# Analisys PALMS Data Set

## Data Processing

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import time

In [ ]:
# select data directory

work_dir = "data/raw/NBBB baseline/"
work_dir = "../test_data/"
acc = "acc/"  # accelerometer data (metadata)
gps = "gps/"  # GPS data (csv format)
merge = "merge/"

# acc and GPS data files have the same name
list_file_acc = sorted(os.listdir(work_dir + acc))
list_file_gps = sorted(os.listdir(work_dir + gps))

# select a file from the list
file_gps = list_file_gps[0]
file_acc = list_file_acc[0]
file_merge = file_acc[:-4] + "-merged.csv"
print("\n Acc data file :   " + file_acc)
print(" GPS data file :   " + file_gps)
print(" Output file   :   " + file_merge + "\n")

In [ ]:
# read GPS data and create a dataframe

path = work_dir + gps + file_gps

columns = pd.read_csv(path, sep=',').columns.values.tolist() # extract headers
columns = [columns[k].strip() for k in range(len(columns))] # remove initial/final spaces

# remove last empty column
gps_df = pd.read_csv(path, sep=',', names=columns).drop(columns=['INDEX'])

# remove duplicate lines to get read of multiple header rows
gps_df = gps_df.drop_duplicates()

# remove the first row which contains a repetition of the header
gps_df = gps_df.drop(gps_df.index[0])
gps_df = gps_df.reset_index()
gps_df = gps_df.drop(columns=['index'])

try:
    gps_df['UTC DATE'] = [gps_df['UTC DATE'][k].strip() for k in range(len(gps_df))] # remove initial/final spaces
    gps_df['UTC TIME'] = [gps_df['UTC TIME'][k].strip() for k in range(len(gps_df))] # remove initial/final spaces
    gps_df['LOCAL DATE'] = [gps_df['LOCAL DATE'][k].strip() for k in range(len(gps_df))] # remove initial/final spaces
    gps_df['LOCAL TIME'] = [gps_df['LOCAL TIME'][k].strip() for k in range(len(gps_df))] # remove initial/final spaces
    # add datetime column
    gps_df['DATETIME'] = pd.to_datetime(gps_df['UTC DATE'] + " " + gps_df['UTC TIME'], format="%Y/%m/%d")
except:
    try:
        gps_df['DATE'] = [gps_df['DATE'][k].strip() for k in range(len(gps_df))] # remove initial/final spaces
        gps_df['TIME'] = [gps_df['TIME'][k].strip() for k in range(len(gps_df))] # remove initial/final spaces
        # add datetime column
        gps_df['DATETIME'] = pd.to_datetime(gps_df['DATE'] + " " + gps_df['TIME'], format="%Y/%m/%d")
    except:
        gps_df['UTC'] = [gps_df['UTC'][k].strip() for k in range(len(gps_df))] # remove initial/final spaces
        gps_df['LOCAL TIME'] = [gps_df['LOCAL TIME'][k].strip() for k in range(len(gps_df))] # remove initial/final spaces
        # add datetime column
        gps_df['DATETIME'] = pd.to_datetime(gps_df['UTC'], format="%Y/%m/%d")

print(gps_df.shape)
#print(gps_df.dtypes)

In [ ]:
gps_df.describe(include='all').transpose()

In [ ]:
gps_df.head(10)

In [ ]:
# Read accelerometer data

path = work_dir + acc + file_acc

acc_list = [line.rstrip('\n') for line in open(path)]

acc_list[0:11]

In [ ]:
# print metadata

start_time = acc_list[2].split()[2]
start_date = acc_list[3].split()[2]
interval = acc_list[4].split()[3]
end_time = acc_list[5].split()[2]
end_date = acc_list[6].split()[2]
print("start time: %s" % start_time)
print("start date: %s" % start_date)
print("time interval: %s" % interval)
print("end time: %s" % end_time)
print("end date: %s" % end_date)

In [ ]:
try:
    start_timestamp = dt.datetime.strptime(start_date + " " + start_time, '%d/%m/%Y %H:%M:%S')
except:
    try:
        start_timestamp = dt.datetime.strptime(start_date + " " + start_time, '%m/%d/%Y %H:%M:%S')
    except:
        start_timestamp = dt.datetime.strptime(start_date + " " + start_time, '%Y/%m/%d %H:%M:%S')
    
x = time.strptime(interval, '%H:%M:%S')
interval = dt.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec)
try:
    end_timestamp = dt.datetime.strptime(end_date + " " + end_time, '%d-%b-%y %H:%M:%S')
except:
    try:
        end_timestamp = dt.datetime.strptime(end_date + " " + end_time, '%d-%m-%Y %H:%M:%S')
    except:
        try:
            end_timestamp = dt.datetime.strptime(end_date + " " + end_time, '%d/%m/%Y %H:%M:%S')
        except:
            try:
                end_timestamp = dt.datetime.strptime(end_date + " " + end_time, '%m/%d/%Y %H:%M:%S')
            except:
                end_timestamp = dt.datetime.strptime(end_date + " " + end_time, '%Y/%m/%d %H:%M:%S')

if len(acc_list[10]) < 50: 
    acc_data = acc_list[10:]
else:
    acc_data = acc_list[11:]
    
tot_intervals = len(acc_data)
print("start timestamp: %s" % start_timestamp)
print("end timestamp: %s" % end_timestamp)
print("interval in sec: %s" % interval.total_seconds())
print("number of acc. events: %d" % tot_intervals)

In [ ]:
# create accelerometer dataframe

acc_df = pd.DataFrame({})
acc_df['DATETIME'] = [start_timestamp + k*interval for k in range(tot_intervals)]
acc_df['ACC DATA'] = acc_data
acc_df.head(10)

In [ ]:
# select gps data based on accelerometer timestamps

start_time = time.clock()
w1 = np.zeros(len(gps_df))
#w2 = np.zeros(len(gps_df))
w3 = gps_df['DATETIME'].copy()
for i in range(len(acc_df)):
    w1 = abs((gps_df['DATETIME'] - acc_df['DATETIME'][i]).dt.total_seconds()) < interval.total_seconds()/2 # only one element equal to 1
    #w2[w1] = 1  # select indices that match the condition in w1; NOT used
    if w1.sum() == 1:  # found a value of gps_df which matches timestamp in acc_df
        w3[w1] = acc_df['DATETIME'][i]
            
print(time.clock() - start_time, "seconds")

In [ ]:
# create merged dataframe

gps_df2 = gps_df.copy()
gps_df2['DATETIME'] = w3
merged_df = pd.merge(acc_df, gps_df2, on='DATETIME')
merged_df.head(10)

In [ ]:
# write merged data frame on file
try:
    os.mkdir(work_dir + merge)
    print("\n Output directory: " + work_dir + merge)
except FileExistsError:
    print("\n Output directory: " + work_dir + merge)


path = work_dir + merge + file_merge
merged_df.to_csv(path)
print(" Output file " + file_merge + " added.")